In [8]:
import os
import sys
import tweepy

ROOT = os.path.abspath('..')
SRC = os.path.join(ROOT, 'src')
os.chdir(ROOT)
sys.path.append(SRC)

from sentiment_analysis.core.utils import load_yaml

In [10]:
os.listdir()

['LICENSE',
 'requirements.txt',
 'tests',
 'README.md',
 'credentials.yaml',
 '.gitignore',
 'scripts',
 '.git',
 'data',
 'notebooks',
 'src']

In [15]:
CREDENTIALS = 'credentials.yaml'
NEWS_SOURCES = os.path.join('data', 'sources.yaml')

credentials = load_yaml(CREDENTIALS)
storage_options = {
    'key': credentials['aws']['access_key'],
    'secret': credentials['aws']['secret_access_key']
}
bearer_token = credentials['twitter']['bearer_token']

news_sources = load_yaml(NEWS_SOURCES, 'sources')
news_sources_id = [s['twitter_id'] for s in news_sources.values()]
news_sources_id

[3108351,
 759251,
 1367531,
 807095,
 2467791,
 17469289,
 20402945,
 7309052,
 15438913,
 742143,
 15754281,
 25589776,
 14511951,
 788524,
 20562637,
 91478624,
 28785486,
 64643056,
 16343974,
 2884771,
 16973333,
 1994321,
 9300262,
 87416722,
 18949452,
 4970411,
 487118986,
 51241574,
 37034483,
 14293310,
 1652541,
 5988062,
 25067168,
 6134882,
 34713362]

In [ ]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta

OFFSET_MIN = 120
end_time = datetime.now(timezone.utc)
start_time = end_time - relativedelta(minutes=OFFSET_MIN)

In [17]:
OFFSET_MIN = 120
end_time = datetime.now(timezone.utc)
start_time = end_time - relativedelta(minutes=OFFSET_MIN)

tweepy_client = tweepy.Client(credentials['twitter']['bearer_token'])

user_id = 3108351
max_results = 30

tw = tweepy_client.get_users_tweets(
    user_id,
    max_results=max_results,
    start_time=start_time,
    end_time=end_time,
    tweet_fields=['public_metrics', 'created_at']
)

In [43]:
import pandas as pd

ds = pd.DataFrame({
    'user_id': [user_id]*len(tw.data),
    'tweet_id': [tweet.id for tweet in tw.data],
    'text': [tweet.text for tweet in tw.data],
    'created_at': [tweet.created_at for tweet in tw.data],
    'public_metrics': [tweet.public_metrics for tweet in tw.data]
})
ds = (
    ds
    .join(ds['public_metrics'].apply(pd.Series))
    .drop(['public_metrics'], axis=1)
)
ds

,user_id,tweet_id,text,created_at,retweet_count,reply_count,like_count,quote_count
0,3108351,1598039102363996178,The Dow rallied more than 300 points after Jer...,2022-11-30 19:39:46+00:00,3,0,6,0
1,3108351,1598036657210593299,"Severe storms spawning tornadoes, hail and hig...",2022-11-30 19:30:03+00:00,7,2,4,0
2,3108351,1598032917325807617,A top European official warned Elon Musk on We...,2022-11-30 19:15:12+00:00,37,44,77,18
3,3108351,1598029124693594122,China’s lockdown protests are the largest show...,2022-11-30 19:00:07+00:00,14,6,30,1
4,3108351,1598025323047682048,Amazon CEO Andy Jassy said he doesn’t regret t...,2022-11-30 18:45:01+00:00,8,4,27,2
5,3108351,1598022434388643850,Breaking: Federal Reserve Chairman Jerome Powe...,2022-11-30 18:33:32+00:00,38,9,108,12
6,3108351,1598020307461279761,From @WSJopinion: For those of us who are free...,2022-11-30 18:25:05+00:00,9,5,38,0
7,3108351,1598017102601912332,Breaking: The House passed a bill to force rai...,2022-11-30 18:12:21+00:00,16,11,32,5
8,3108351,1598014110083448845,"Big companies have been cutting jobs, even as ...",2022-11-30 18:00:28+00:00,12,2,10,0
9,3108351,1598011505055760385,Who gets the jury summons? Or the email addres...,2022-11-30 17:50:07+00:00,5,7,35,3


In [58]:
12 * 100 * 12 * 30 * ds.memory_usage(deep=True).sum()/1024/1024

1725.8148193359375

In [50]:
(ds.memory_usage(deep=True).sum()/1024/1024)*35*12

1.6778755187988281

In [53]:
40*35*12*30

504000